In [1]:
import numpy as np
import matplotlib as mpl
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Dropout
from tensorflow.keras import optimizers
from sklearn.model_selection import KFold
import csv

mpl.use('Agg')
import matplotlib.pyplot as plt


In [2]:
#df1 = pd.read_csv('weather_train.csv', sep=';', header=None)
#df2 = pd.read_csv('weather_test.csv', sep=';',header=None)
#df3 = pd.read_csv('weather_dev.csv', sep=';', header=None)
    
# Concatenate the DataFrames vertically
#merged_df = pd.concat([df1, df2, df3])
                          


#column_names = ['hour', 'day', 'month', 'year', 'cloud_coverage', 
               # 'Visibility', 'temperature', 'dew_point', 'relative_humidity', 'wind_speed', 
               # 'station_pressure', 'altimeter', 'solar_energy']
#merged_df.columns = column_names
#merged_df.head()

#merged_df = merged_df.sort_values(by = ['year', 'month', 'day', 'hour'])
#merged_df.head()

,hour,day,month,year,cloud_coverage,Visibility,temperature,dew_point,relative_humidity,wind_speed,station_pressure,altimeter,solar_energy
2792,6,1,2,2016,0.00,9.92,0.37,-0.01,89.12,4.72,29.19,29.98,0.00
4682,7,1,2,2016,0.00,10.00,0.47,-0.04,90.08,6.00,29.20,29.99,84.29
1947,8,1,2,2016,0.14,9.92,0.89,0.30,92.00,2.76,29.22,30.01,531.00
1846,9,1,2,2016,0.92,10.00,1.97,0.41,86.12,5.56,29.24,30.04,923.75
724,10,1,2,2016,0.46,10.00,3.15,1.07,82.48,2.12,29.27,30.06,1947.75


In [13]:
merged_df.dtypes

hour                   int64
day                    int64
month                  int64
year                   int64
cloud_coverage       float64
Visibility           float64
temperature          float64
dew_point            float64
relative_humidity    float64
wind_speed           float64
station_pressure     float64
altimeter            float64
solar_energy         float64
dtype: object

In [3]:
#total_length = len(merged_df)
#train_end = int(total_length * 0.8)
#dev_end = train_end + int(total_length * 0.1)

# Perform the split
#train_df = merged_df[:train_end]
#dev_df = merged_df[train_end:dev_end]
#test_df = merged_df[dev_end:]

# Verify the splits
#print(f"Training data: {train_df.shape}")
#print(f"Development data: {dev_df.shape}")
#print(f"Test data: {test_df.shape}")

# Optional: Save to CSV if needed
#train_df.to_csv('weather_train.csv', index=False)
#dev_df.to_csv('weather_dev.csv', index=False)
#test_df.to_csv('weather_test.csv', index=False)


Training data: (6028, 13)
Development data: (753, 13)
Test data: (755, 13)


In [4]:
# Set y values of data to lie between 0 and 1
def normalize_data(dataset, data_min, data_max):
    data_std = (dataset - data_min) / (data_max - data_min)
    test_scaled = data_std * (np.amax(data_std) - np.amin(data_std)) + np.amin(data_std)
    return test_scaled


In [5]:
# Import and pre-process data for future applications
def import_data(train_dataframe, dev_dataframe, test_dataframe):
    dataset = train_dataframe.values
    dataset = dataset.astype('float32')

    # Include all 12 initial factors (Year ; Month ; Hour ; Day ; Cloud Coverage ; Visibility ; Temperature ; Dew Point ;
    # Relative Humidity ; Wind Speed ; Station Pressure ; Altimeter
    max_test = np.max(dataset[:, 12])
    min_test = np.min(dataset[:, 12])
    scale_factor = max_test - min_test
    max = np.empty(13)
    min = np.empty(13)

    # Create training dataset
    for i in range(0, 13):
        min[i] = np.amin(dataset[:, i], axis=0)
        max[i] = np.amax(dataset[:, i], axis=0)
        dataset[:, i] = normalize_data(dataset[:, i], min[i], max[i])

    train_data = dataset[:, 0:12]
    train_labels = dataset[:, 12]

    # Create dev dataset
    dataset = dev_dataframe.values
    dataset = dataset.astype('float32')

    for i in range(0, 13):
        dataset[:, i] = normalize_data(dataset[:, i], min[i], max[i])

    dev_data = dataset[:, 0:12]
    dev_labels = dataset[:, 12]

    # Create test dataset
    dataset = test_dataframe.values
    dataset = dataset.astype('float32')

    for i in range(0, 13):
        dataset[:, i] = normalize_data(dataset[:, i], min[i], max[i])

    test_data = dataset[:, 0:12]
    test_labels = dataset[:, 12]

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels, scale_factor


In [6]:
 #Construct and return Keras RNN model
def build_model(init_type='glorot_uniform', optimizer='adam'):
    model = Sequential()
    layers = [12, 64, 64, 1, 1]
    model.add(LSTM(
        layers[0],
        input_shape=(None, 12),
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        layers[1],
        kernel_initializer=init_type,
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(Dense(
        layers[2], activation='tanh',
        kernel_initializer=init_type))
    model.add(Dense(layers[3]))

    model.add(Activation("relu"))

    rms = optimizers.RMSprop(learning_rate=0.002, rho=0.9, epsilon=1e-08)
    model.compile(loss="mean_squared_error", optimizer=optimizer)

    return model






In [7]:
# Save output predictions for graphing and inspection
def write_to_csv(prediction, filename):
    print("Writing to CSV...")
    with open(filename, 'w') as file:
        for i in range(prediction.shape[0]):
            file.write("%.5f" % prediction[i][0][0])
            file.write('\n')
    print("...finished!")


In [8]:
# Return MSE error values of all three data sets based on a single model
def evaluate(model, X_train, Y_train, X_dev, Y_dev, X_test, Y_test, scale_factor):
    scores = model.evaluate(X_train, Y_train, verbose=0) * scale_factor * scale_factor
    print("train: ", model.metrics_names, ": ", scores)
    scores = model.evaluate(X_dev, Y_dev, verbose=0) * scale_factor * scale_factor
    print("dev: ", model.metrics_names, ": ", scores)
    scores = model.evaluate(X_test, Y_test, verbose=0) * scale_factor * scale_factor
    print("test: ", model.metrics_names, ": ", scores)


In [9]:
# Calculate MSE between two arrays of values
def mse(predicted, observed):
    return np.sum(np.multiply((predicted - observed), (predicted - observed))) / predicted.shape[0]


In [14]:
def main():
    plt.switch_backend('tkAgg')

    # Import test data (6027, 13)
    train_dataframe = pd.read_csv('weather_train.csv')
    dev_dataframe = pd.read_csv('weather_dev.csv')
    test_dataframe = pd.read_csv('weather_test.csv')
    train_data, train_labels, dev_data, dev_labels, test_data, test_labels, scale_factor = import_data(train_dataframe, dev_dataframe, test_dataframe)

    X_train = np.reshape(train_data, (train_data.shape[0], 1, train_data.shape[1]))
    X_dev = np.reshape(dev_data, (dev_data.shape[0], 1, dev_data.shape[1]))
    X_test = np.reshape(test_data, (test_data.shape[0], 1, test_data.shape[1]))
    Y_train = np.reshape(train_labels, (train_labels.shape[0], 1, 1))
    Y_dev = np.reshape(dev_labels, (dev_labels.shape[0], 1, 1))
    Y_test = np.reshape(test_labels, (test_labels.shape[0], 1, 1))

    model = build_model('glorot_uniform', 'adam')

    # Standard vanilla LSTM model
    model_fit_epochs = 100
    print("X_train shape: ", X_train.shape, " Y_train shape: ", Y_train.shape)

    model.fit(X_train, Y_train, batch_size=16, epochs=model_fit_epochs)
    trainset_predicted = model.predict(X_train)
    devset_predicted = model.predict(X_dev)
    testset_predicted = model.predict(X_test)

    print("Train MSE: ", mse(trainset_predicted, Y_train) * scale_factor * scale_factor)
    print("Dev MSE: ", mse(devset_predicted, Y_dev) * scale_factor * scale_factor)
    print("Test MSE: ", mse(testset_predicted, Y_test) * scale_factor * scale_factor)

    # K-fold cross validation (K = 10):
    kf = KFold(n_splits=10, shuffle=True)
    # Loop through the indices the split() method returns
    for index, (train_indices, val_indices) in enumerate(kf.split(X_train, Y_train)):
        print("Training on fold " + str(index + 1) + "/10...")
        # Generate batches from indices
        xtrain, xval = X_train[train_indices], X_train[val_indices]
        ytrain, yval = Y_train[train_indices], Y_train[val_indices]
        # Clear model, and create it
        model = None
        model = build_model()

        model.fit(xtrain, ytrain, batch_size=16, epochs=model_fit_epochs)
        testset_predicted = model.predict(xval)
        print("Test MSE: ", mse(testset_predicted, yval))
        
# Grid search to optimize model params

    init = ['glorot_uniform', 'normal', 'uniform']
    epochs = [50, 100, 150]
    batches = [8, 16, 32]
    optimizers = ['rmsprop', 'adam']
    optimal_params = np.empty(4)
    minimum_error = 2e63

    for init_type in init:
        for epoch in epochs:
            for batch in batches:
                for optimizer in optimizers:
                    model = None
                    model = build_model(init_type, optimizer)

                    model.fit(X_train, Y_train, batch_size=batch, epochs=epoch)
                    predicted = model.predict(X_test)
                    error = mse(predicted, Y_test)
                    if error < minimum_error:
                        minimum_error = error
                        optimal_params = [init_type, epoch, batch, optimizer]

    print("optimal params: ", optimal_params)
    print("minimized error: ", minimum_error)
    
    write_to_csv(trainset_predicted, 'nn_trainset_prediction.csv')
    write_to_csv(devset_predicted, 'nn_devset_prediction.csv')
    write_to_csv(testset_predicted, 'nn_testset_prediction.csv')

    return



In [15]:
if __name__ == '__main__':
    main()

X_train shape:  (6028, 1, 12)  Y_train shape:  (6028, 1, 1)
Epoch 1/100


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


377/377 ━━━━━━━━━━━━━━━━━━━━ 1s 948us/step - loss: 0.1000
Epoch 2/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - loss: 0.0614
Epoch 3/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0580
Epoch 4/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.0581
Epoch 5/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - loss: 0.0535
Epoch 6/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - loss: 0.0493
Epoch 7/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - loss: 0.0435
Epoch 8/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - loss: 0.0400
Epoch 9/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - loss: 0.0366
Epoch 10/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - loss: 0.0372
Epoch 11/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - loss: 0.0346
Epoch 12/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - loss: 0.0337
Epoch 13/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 0.0328
Epoch 14/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0314
Epoch 15/100
377/377 ━━━━━━━━━━

377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0276
Epoch 80/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0262
Epoch 81/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - loss: 0.0271
Epoch 82/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0260
Epoch 83/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0250
Epoch 84/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0261
Epoch 85/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - loss: 0.0273
Epoch 86/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 970us/step - loss: 0.0277
Epoch 87/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - loss: 0.0265
Epoch 88/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - loss: 0.0267
Epoch 89/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - loss: 0.0269
Epoch 90/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0264
Epoch 91/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - loss: 0.0259
Epoch 92/100
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - loss: 0.0255
Epoch 93/100
377/377 ━━━━━━━━━━

340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - loss: 0.0293
Epoch 55/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.0282
Epoch 56/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - loss: 0.0276
Epoch 57/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - loss: 0.0286
Epoch 58/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - loss: 0.0276
Epoch 59/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - loss: 0.0287
Epoch 60/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - loss: 0.0283
Epoch 61/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - loss: 0.0282
Epoch 62/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - loss: 0.0271
Epoch 63/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - loss: 0.0269
Epoch 64/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - loss: 0.0292
Epoch 65/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - loss: 0.0285
Epoch 66/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - loss: 0.0277
Epoch 67/100
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - loss: 0.0274
Epoch 68/100
340/34

...

377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - loss: 0.0249
Epoch 135/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - loss: 0.0248
Epoch 136/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 908us/step - loss: 0.0254
Epoch 137/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - loss: 0.0252
Epoch 138/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step - loss: 0.0261
Epoch 139/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - loss: 0.0245
Epoch 140/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 816us/step - loss: 0.0250
Epoch 141/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - loss: 0.0257
Epoch 142/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - loss: 0.0239
Epoch 143/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - loss: 0.0249
Epoch 144/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - loss: 0.0242
Epoch 145/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - loss: 0.0250
Epoch 146/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - loss: 0.0249
Epoch 147/150
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 0.0252
Epoch 

189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - loss: 0.0278
Epoch 62/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - loss: 0.0278
Epoch 63/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - loss: 0.0276
Epoch 64/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - loss: 0.0286
Epoch 65/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - loss: 0.0278
Epoch 66/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - loss: 0.0280
Epoch 67/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - loss: 0.0280
Epoch 68/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - loss: 0.0274
Epoch 69/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - loss: 0.0279
Epoch 70/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - loss: 0.0276
Epoch 71/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - loss: 0.0281
Epoch 72/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - loss: 0.0296
Epoch 73/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - loss: 0.0273
Epoch 74/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - loss: 0.0277
Epoch 75/150
189/18

189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 0.0272
Epoch 139/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - loss: 0.0271
Epoch 140/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - loss: 0.0253
Epoch 141/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - loss: 0.0270
Epoch 142/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step - loss: 0.0260
Epoch 143/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - loss: 0.0278
Epoch 144/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - loss: 0.0267
Epoch 145/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - loss: 0.0265
Epoch 146/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - loss: 0.0269
Epoch 147/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - loss: 0.0276
Epoch 148/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - loss: 0.0256
Epoch 149/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0258
Epoch 150/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0270
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Epoch 1/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s

189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.0279
Epoch 66/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step - loss: 0.0280
Epoch 67/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - loss: 0.0284
Epoch 68/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - loss: 0.0292
Epoch 69/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - loss: 0.0268
Epoch 70/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - loss: 0.0259
Epoch 71/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - loss: 0.0277
Epoch 72/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - loss: 0.0275
Epoch 73/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - loss: 0.0281
Epoch 74/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step - loss: 0.0272
Epoch 75/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0265
Epoch 76/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - loss: 0.0270
Epoch 77/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 956us/step - loss: 0.0263
Epoch 78/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - loss: 0.0271
Epoch 79/150
189/189 

189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - loss: 0.0278
Epoch 144/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - loss: 0.0258
Epoch 145/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - loss: 0.0265
Epoch 146/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - loss: 0.0259
Epoch 147/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - loss: 0.0269
Epoch 148/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - loss: 0.0257
Epoch 149/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - loss: 0.0258
Epoch 150/150
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - loss: 0.0257
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
optimal params:  ['normal', 150, 32, 'adam']
minimized error:  0.014688849291264617
Writing to CSV...
...finished!
Writing to CSV...
...finished!
Writing to CSV...
...finished!
